# PFRA 

### _Development Notebook_
--- 

Tests using scrapbook

In [10]:
import sys;sys.path.append('../')

In [190]:
from hecrasio.core import *
from hecrasio.qaqc import *
from hecrasio.logger import *
import scrapbook as sb
import pandas as pd
import json
import shutil
from collections import ChainMap

In [41]:
# Make copies of example_nb
example_nb = "SB-QAQC-FluvialTest.ipynb"
for i in range(10):
    shutil.copy(example_nb , os.path.join(os.getcwd(), "temp", "SB-QAQC-FluvialTest-{}.ipynb".format(i) ))

In [46]:
# Create a large list of notebooks
nbs = os.listdir(os.path.join(os.getcwd(), "temp"))
nbs[0]

'SB-QAQC-FluvialTest-0.ipynb'

In [ ]:
# Read many notebooks
book = sb.read_notebooks(os.path.join(os.getcwd(), "temp"))

In [193]:
# Create a unified results dictionary
results_dict = {}
for tup in list(book.items()):
    nb, results = tup
    result_dict = dict(ChainMap(*[list(json.loads(scrap).values())[0] for scrap in results.scraps]))
    results_dict[nb] = result_dict

In [197]:
# Pull results into a large review table
results_table = pd.DataFrame.from_dict(results_dict).T
results_table.head(3)

,1D Cores,1D Methodology,1D2D Flow Tolerance,1D2D MaxIter,1D2D Minimum Flow Tolerance,1D2D WS Tolerance,2D Boundary Condition Ramp Up Fraction,2D Boundary Condition Volume Check,2D Cores,2D Coriolis,...,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
SB-QAQC-FluvialTest-0,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0
SB-QAQC-FluvialTest-1,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0
SB-QAQC-FluvialTest-2,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,...,0,99553.8,84515.9,0,36.8438,0.00171806,0,0,0,0


In [320]:
df = pd.DataFrame(columns=['Unique_Values', 'Warnings', 'Offending_Nbs'])
df['Result_Attribute'] = pd.Index(list(results_table.columns))
df.set_index('Result_Attribute', drop=True, inplace=True)
df.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,NaN,NaN,NaN
1D Methodology,NaN,NaN,NaN
1D2D Flow Tolerance,NaN,NaN,NaN
1D2D MaxIter,NaN,NaN,NaN
1D2D Minimum Flow Tolerance,NaN,NaN,NaN


In [321]:
for i in df.index:
    dtype = type(results_table[i][0])
    if dtype is int or dtype is float:
        df.loc[i]['Unique_Values'] = list(np.unique(results_table[i]))
    elif dtype is str:
        df.loc[i]['Unique_Values'] = list(set(results_table[i]))
    elif dtype is list:
        series_list = [elem[0] for elem in results_table[i]]
        list_dtype = type(series_list[0])
        if list_dtype is int or list_dtype is float:
            df.loc[i]['Unique_Values'] = list(np.unique(series_list))
        elif list_dtype is str:
            df.loc[i]['Unique_Values'] = list(set(series_list))
        else:
            print("Dtype {} is not currently supported for variable {}".format(dtype, i))
    else:
        print("Dtype {} is not currently supported for variable {}".format(dtype, i))
df.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],NaN,NaN
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN


In [355]:
# Example warnings
for i in df.index:
    values = df.loc[i][0]
    if i == '1D Cores':
        t1 = 0
        df.loc[i]['Warnings'] = 'WARNING' if any([value > t1 for value in values]) else 'PASS'
        df.loc[i]['Offending_Nbs'] = [results_table.index[i] for i, value in enumerate(list(results_table[i])) if value > t1]
    elif i == 'Vol Accounting Error':
        t2 = 30
        df.loc[i]['Warnings'] = 'WARNING' if any([value > t2 for value in values]) else 'PASS'
        df.loc[i]['Offending_Nbs'] = [results_table.index[i] for i, value in enumerate(list(results_table[i])) if value > t2]
    elif i == 'Vol Accounting Error Percentage':
        t3 = 0.001
        df.loc[i]['Warnings'] = 'WARNING' if any([value > t3 for value in values]) else 'PASS'
        df.loc[i]['Offending_Nbs'] = [results_table.index[i] for i, value in enumerate(list(results_table[i])) if value > t3]
df.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],PASS,[]
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN


In [356]:
df

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],PASS,[]
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN
1D2D WS Tolerance,[0.0099999998],NaN,NaN
2D Boundary Condition Ramp Up Fraction,[0.1000000015],NaN,NaN
2D Boundary Condition Volume Check,[False],NaN,NaN
2D Cores,[0],NaN,NaN


In [357]:
# Functionalize the above
def validate(pd_df, attr, value_list, threshold, results_table_df):
    """Validate the results table raising warnings if any values reported in the data frame are above a given
    value. Report which notebooks are above that value.
    """
    pd_df.loc[attr]['Warnings'] = 'WARNING' if any([value > threshold for value in value_list]) else 'PASS'
    pd_df.loc[attr]['Offending_Nbs'] = [results_table_df.index[i] for i, value in enumerate(list(results_table_df[attr])) if value > threshold]

In [362]:
# Example warnings
test_df = df.copy()

for i in test_df.index:
    values = test_df.loc[i][0]
    if i == '1D Cores':
        validate(test_df, i, values, 0, results_table)
    elif i == 'Vol Accounting Error':
        validate(test_df, i, values, 30, results_table)
    elif i == 'Vol Accounting Error Percentage':
        validate(test_df, i, values, 0.001, results_table)
test_df.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
1D Cores,[0],PASS,[]
1D Methodology,[Finite Difference],NaN,NaN
1D2D Flow Tolerance,[0.1000000015],NaN,NaN
1D2D MaxIter,[0],NaN,NaN
1D2D Minimum Flow Tolerance,[1.0],NaN,NaN


# END